In [1]:
import pandas as pd 
import numpy as np 
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt 
import distance
import json
import os
import slack
from datetime import timedelta,datetime
from sshtunnel import SSHTunnelForwarder
import psycopg2
import nbslack
import seaborn as sns
import gender_guesser.detector as gender
from ethnicolr import census_ln, pred_census_ln
# alloy middesk distribution

//anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [7]:
webhook = 'https://hooks.slack.com/services/' + str(os.environ.get('SLACK_ANALYTICS_WEB'))
nbslack.notifying('Siddhesvar',webhook)

In [2]:
try:
    with SSHTunnelForwarder(
        (os.environ.get('JOUST_IP'), 22),
        ssh_private_key="~/.ssh/id_rsa",
        ssh_username="ubuntu",
        remote_bind_address=(os.environ.get('JOUST_ADDRESS'), 5432)) as server:

        server.start()
        print("server connected")

        params = {
            "database": "joust_production",
            "user": "sid",
            "password": os.environ.get('JOUST_PASS'),
            "host": "localhost",
            "port": server.local_bind_port
        }

        conn = psycopg2.connect(**params)
        curs = conn.cursor()
        print("database connected")

        #curs.execute("select * from public.account_holders")
        transfers = "select account_holders.id, amount, transfers.created_at as \"created_at.1\" from account_holders inner join transfers on account_holders.id = transfers.account_holder_id"
        total = "select account_holders.id, account_holders.created_at, account_holders.archived, account_activations.status, account_holders.email, first_name, last_name, verification_status as q2_core_pro_verification_status, devices.platform from account_holders inner join account_holder_devices on account_holders.id = account_holder_devices.account_holder_id inner join devices on account_holder_devices.device_id = devices.id left join core_pro_customers on account_holders.id = core_pro_customers.account_holder_id left join account_activations on account_holders.id = account_activations.account_holder_id"
        invoices = "select account_holders.id, status, amount, account_holders.created_at, payment_requests.created_at as \"created_at.1\", accepted_date, payment_requests.paid_date from account_holders inner join payment_requests on account_holders.id = payment_requests.account_holder_id"
        pcodes = "SELECT core_pro_customers.account_holder_id as id, promo_codes.\"name\" as promo_codes from core_pro_customers left join account_holder_promo_codes on core_pro_customers.account_holder_id = account_holder_promo_codes.account_holder_id left join promo_codes on promo_codes.id = account_holder_promo_codes.promo_code_id"
        #total2 = "select account_holders.id, account_holders.created_at, first_name, last_name, q2_core_pro_verification_status from account_holders"


        #result = curs.fetchall()
        tra = pd.read_sql_query(transfers, conn)
        ids = pd.read_sql_query(total, conn)
        req = pd.read_sql_query(invoices, conn)
        #ids2 = pd.read_sql_query(total2, conn)
        promos = pd.read_sql_query(pcodes, conn)

        conn.close()

except Exception as e:
    print(e)
    print("Connection Failed")

server connected
database connected


In [3]:
fin = ids
fin['created_at'] = pd.to_datetime(fin['created_at'])
fin['created_at'] = fin['created_at'].dt.tz_localize('utc')
fin['created_at'] = fin['created_at'].dt.tz_convert('US/Central')
fin.fillna(value='unregistered', inplace=True)

In [4]:
yest = datetime.utcnow() - timedelta(1)
yr = yest.year
mo = yest.month
dy = yest.day

In [5]:
fin['Year'] = fin['created_at'].dt.year
fin['Month'] = fin['created_at'].dt.month
fin['Day'] = fin['created_at'].dt.day

In [6]:
fin2 = fin[fin['Year'] == yr]
fin2 = fin2[fin2['Month'] == mo]
fin2 = fin2[fin2['Day'] == dy]
fin2.drop_duplicates(subset ="id", keep = False, inplace = True)

,id,created_at,archived,status,email,first_name,last_name,q2_core_pro_verification_status,platform,Year,Month,Day


In [13]:
ios = len(fin2[fin2['platform'] == 'ios'])
android = len(fin2[fin2['platform'] == 'android'])
oth = len(fin2) - ios - android

In [14]:
nbslack.notify('Yesterday, ' + str(len(fin2)) + ' people downloaded the app. ' + '(ios: ' + str(ios) + ', android: ' + str(android) +')')


In [15]:
minval = fin2['id'].min()
pcr = ''
promos2 = promos[promos['id']>minval].dropna()
upcodes = np.sort(promos2['promo_codes'].unique())
for k in upcodes:
    pcr = pcr + (str(len(promos2[promos2['promo_codes'] == k])) + ' ' + k + ', ')
if len(upcodes) > 0:
    nbslack.notify('Out of those '+ str(len(fin2)) +' people, we saw ' + pcr[:-2])

In [16]:
nbslack.notify('Out of those '+ str(len(fin2)) +' people, ' + str(len(fin2[fin2['q2_core_pro_verification_status'] != 'unregistered'])) + ' (or ' + str(round(100*len(fin2[fin2['q2_core_pro_verification_status'] != 'unregistered'])/len(fin2),2)) + '%) finished the sign up process.')
#nbslack.notify('Out of those '+ str(len(fin2)) +' people, ' + str(len(fin2[fin2['q2_core_pro_verification_status'] == 'Denied'])) + ' (or ' + str(round(100*len(fin2[fin2['q2_core_pro_verification_status'] == 'Denied'])/len(fin2),2)) + '%) got denied.')
fin2 = fin2[fin2['q2_core_pro_verification_status'] != 'unregistered']

In [17]:
nbslack.notify('Out of those '+ str(len(fin2)) +' people, ' + str(len(fin2[fin2['q2_core_pro_verification_status'] == 'verified'])) + ' (or ' + str(round(100*len(fin2[fin2['q2_core_pro_verification_status'] == 'verified'])/len(fin2),2)) + '%) got approved.')
nbslack.notify('Out of those '+ str(len(fin2)) +' people, ' + str(len(fin2[fin2['q2_core_pro_verification_status'] == 'denied'])) + ' (or ' + str(round(100*len(fin2[fin2['q2_core_pro_verification_status'] == 'denied'])/len(fin2),2)) + '%) got denied.')



In [18]:
ids = fin2['id'].unique()
ctreq = 0
cttra = 0
for k in ids:
    if (len(req[req['id'] == k]) > 0):
        ctreq = ctreq + 1
    if (len(tra[tra['id'] == k]) > 0):
        cttra = cttra + 1
print(ctreq)
print(cttra)

1
0


In [19]:
nbslack.notify('Out of those '+ str(len(fin2[fin2['q2_core_pro_verification_status'] == 'verified'])) +' people, ' + str(ctreq) + ' (or ' + str(round(100*ctreq/len(fin2[fin2['q2_core_pro_verification_status'] == 'verified']),2)) + '%) sent atleast 1 invoice.')
nbslack.notify('Out of those '+ str(len(fin2[fin2['q2_core_pro_verification_status'] == 'verified'])) +' people, ' + str(cttra) + ' (or ' + str(round(100*cttra/len(fin2[fin2['q2_core_pro_verification_status'] == 'verified']),2)) + '%) made atleast 1 bank transfer.')

In [20]:
req['paid_date'] = req['paid_date'].dt.tz_localize('utc')
req['paid_date'] = req['paid_date'].dt.tz_convert('US/Central')
req['Year'] = req['paid_date'].dt.year
req['Month'] = req['paid_date'].dt.month
req['Day'] = req['paid_date'].dt.day
req.head()

,id,status,amount,created_at,created_at.1,accepted_date,paid_date,Year,Month,Day
0,7,0,4.0,2018-12-06 16:26:45.573220,2019-01-21 18:42:17.241,NaT,NaT,NaN,NaN,NaN
1,9,0,2.0,2018-12-07 23:39:37.223218,2019-01-16 00:49:51.166,NaT,NaT,NaN,NaN,NaN
2,10,3,1.6,2018-12-08 01:34:16.491605,2019-04-15 23:05:30.538,NaT,NaT,NaN,NaN,NaN
3,9,4,1000.0,2018-12-07 23:39:37.223218,2019-02-09 00:07:59.685,2019-02-09 00:08:48.880,NaT,NaN,NaN,NaN
4,9,0,50.0,2018-12-07 23:39:37.223218,2019-02-07 18:17:24.584,NaT,NaT,NaN,NaN,NaN


In [21]:
req2 = req[req['Year'] == yr]
req2 = req2[req2['Month'] == mo]
req2 = req2[req2['Day'] == dy]
req2

,id,status,amount,created_at,created_at.1,accepted_date,paid_date,Year,Month,Day
172,9,1,20.0,2018-12-07 23:39:37.223218,2020-03-12 16:43:43.625214,2020-03-12 16:46:55.340340,2020-04-07 16:50:25.222133-05:00,2020.0,4.0,7.0
393,4874,1,1.0,2020-03-01 00:56:07.428152,2020-03-29 21:48:36.047343,2020-04-01 10:06:56.661331,2020-04-07 09:05:10.409629-05:00,2020.0,4.0,7.0


In [22]:
pinv = req2[req2['status'] == 1]
nbslack.notify(str(len(pinv)) +' invoices were paid yesterday for a total amount value of $' + str(sum(pinv['amount'])))

In [23]:
req.head()

,id,status,amount,created_at,created_at.1,accepted_date,paid_date,Year,Month,Day
0,7,0,4.0,2018-12-06 16:26:45.573220,2019-01-21 18:42:17.241,NaT,NaT,NaN,NaN,NaN
1,9,0,2.0,2018-12-07 23:39:37.223218,2019-01-16 00:49:51.166,NaT,NaT,NaN,NaN,NaN
2,10,3,1.6,2018-12-08 01:34:16.491605,2019-04-15 23:05:30.538,NaT,NaT,NaN,NaN,NaN
3,9,4,1000.0,2018-12-07 23:39:37.223218,2019-02-09 00:07:59.685,2019-02-09 00:08:48.880,NaT,NaN,NaN,NaN
4,9,0,50.0,2018-12-07 23:39:37.223218,2019-02-07 18:17:24.584,NaT,NaT,NaN,NaN,NaN


In [19]:
0 * * * * eval `ssh-agent -s` && ssh-add && //anaconda3/bin/python /Users/siddhesvark/Desktop/Data\ Analysis/Slack\ Flags/Slack\ experimentation\ 2.py
chmod +x /Users/siddhesvark/Desktop/Data\ Analysis/Slack\ Flags/Slack\ experimentation\ 2.py

SyntaxError: invalid syntax (<ipython-input-19-ff3a141c9e91>, line 1)

In [30]:
date = []
downloads = []
iosct = []
androidct = []
signup = []
verified = []
denied = []
invoice = []
transfer = []
sinv = []
minv = []
for m in range(2):
    for k in range(31):
        fin2 = fin[fin['Year'] == 2020]
        fin2 = fin2[fin2['Month'] == (m+1)]
        fin2 = fin2[fin2['Day'] == (k+1)]
        ios = len(fin2[fin2['platform'] == 'ios'])
        android = len(fin2[fin2['platform'] == 'android'])
        date.append(str(m+1) + '/' + str(k+1) + '/2020')
        downloads.append(len(fin2))
        iosct.append(ios)
        androidct.append(android)
        signup.append(len(fin2[fin2['q2_core_pro_verification_status'] != 'unregistered']))
        verified.append(len(fin2[fin2['q2_core_pro_verification_status'] == 'verified']))
        denied.append(len(fin2[fin2['q2_core_pro_verification_status'] == 'denied']))
        fin2 = fin2[fin2['q2_core_pro_verification_status'] != 'unregistered']
        ids = fin2['id'].unique()
        ctreq = 0
        cttra = 0
        for k2 in ids:
            if (len(req[req['id'] == k2]) > 0):
                ctreq = ctreq + 1
            if (len(tra[tra['id'] == k2]) > 0):
                cttra = cttra + 1
        invoice.append(ctreq)
        transfer.append(cttra)
        req2 = req[req['Year'] == 2020]
        req2 = req2[req2['Month'] == m+1]
        req2 = req2[req2['Day'] == k+1]
        pinv = req2[req2['status'] == 1]
        sinv.append(len(pinv))
        minv.append(sum(pinv['amount']))

In [31]:
fb = pd.DataFrame(date, columns=['Date'])
fb['downloads'] = downloads
fb['ios'] = iosct
fb['android'] = androidct
fb['signup fin'] = signup
fb['verified'] = verified
fb['denied'] = denied
fb['invoicers'] = invoice
fb['transferer'] = transfer
fb['invoices paid'] = sinv
fb['paid amount'] = minv
#fb = fb.set_index('Date')
#fb.to_csv('2020DailyKPI.csv')
fb

,Date,downloads,ios,android,signup fin,verified,denied,invoicers,transferer,invoices paid,paid amount
0,1/1/2020,3,2,1,3,1,1,0,0,5,1900.00
1,1/2/2020,4,1,3,2,0,0,0,0,0,0.00
2,1/3/2020,8,5,3,6,5,0,1,1,1,1900.00
3,1/4/2020,9,3,6,6,2,1,1,0,0,0.00
4,1/5/2020,16,7,9,11,6,1,1,3,0,0.00
5,1/6/2020,17,13,4,11,6,3,1,1,0,0.00
6,1/7/2020,13,9,4,9,6,2,2,1,0,0.00
7,1/8/2020,9,7,2,7,4,1,0,0,0,0.00
8,1/9/2020,11,10,1,8,6,0,2,1,0,0.00
9,1/10/2020,15,11,4,8,8,0,4,1,1,950.00


In [32]:
fb = fb[:-2]
fb['Date'] = pd.to_datetime(fb['Date'])
fb['Week'] = fb['Date'].dt.week
fb

W0223 14:54:59.232609 4656264640 warnings.py:110] //anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  

W0223 14:54:59.310403 4656264640 warnings.py:110] //anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until



,Date,downloads,ios,android,signup fin,verified,denied,invoicers,transferer,invoices paid,paid amount,Week
0,2020-01-01,3,2,1,3,1,1,0,0,5,1900.00,1
1,2020-01-02,4,1,3,2,0,0,0,0,0,0.00,1
2,2020-01-03,8,5,3,6,5,0,1,1,1,1900.00,1
3,2020-01-04,9,3,6,6,2,1,1,0,0,0.00,1
4,2020-01-05,16,7,9,11,6,1,1,3,0,0.00,1
5,2020-01-06,17,13,4,11,6,3,1,1,0,0.00,2
6,2020-01-07,13,9,4,9,6,2,2,1,0,0.00,2
7,2020-01-08,9,7,2,7,4,1,0,0,0,0.00,2
8,2020-01-09,11,10,1,8,6,0,2,1,0,0.00,2
9,2020-01-10,15,11,4,8,8,0,4,1,1,950.00,2


In [34]:
#fb = fb.drop(['Date'], axis=1)
fb.groupby(['Week']).sum().to_csv('2020WeeklyKPI.csv')